+ 1.读取训练集和测试集数据进行预处理
+ 2.训练逻辑回归模型
+ 3.保存模型
+ 4.读取基因数据（基因数据不全，经过预处理大概16000个）
+ 5.定义特征计算函数(继续优化)
+ 6.封装预测函数（输入需要预测的trf序列进行特征计算）
+ 7.调用保存的模型进行预测
+ 8.输出靶标基因并写入csv文件中

+ 修改（增加预测参数，选择靶基因的结合区：3' or 5' or cds）


In [ ]:
# 加载数据预处理
#upload data and package
import pandas as pd
import mglearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

trf_train_data = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\train_data_3.csv")
trf_test_data = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\test_data_3.csv")



In [ ]:
#设置训练集和测试集数据
train_data = trf_train_data[trf_train_data.columns[9:16]]
train_data = np.array(train_data).tolist()

test_data = trf_test_data[trf_test_data.columns[9:16]]
test_data = np.array(test_data).tolist()

In [ ]:
#设置标签数据
train_data_label = np.array(trf_train_data).tolist()
train_data_label = [i[16] for i in train_data_label]

test_data_label = np.array(trf_test_data).tolist()
test_data_label = [i[16] for i in test_data_label]

In [ ]:
X_train = train_data
X_test = test_data
y_train = train_data_label
y_test = test_data_label

In [ ]:
#使用逻辑回归模型(0.1)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5,shuffle=True, random_state=0)
#正则化系数C = 1时(可调节参数C)
logreg = LogisticRegression(C = 0.001).fit(X_train, y_train)


print("Training set score: {:.3f}".format(logreg.score(X_train, y_train)))
print("Test set score: {:.3f}".format(logreg.score(X_test, y_test)))
print("The feature weight: {}".format(logreg.coef_))

In [ ]:
#保存模型
import joblib
joblib.dump(logreg,"E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\logreg.pkl")

In [ ]:
#读取基因列表
gene_list = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\unique_gene.csv")
gene_list = gene_list.iloc[:,1:5]

In [ ]:
#定义特征计算函数

#1.创建一个trf序列的列表
def create(trf):
    seq = [trf]
    trf_seq = pd.DataFrame(seq*16606)
    trf_seq.columns = ["trf_seq"]
    return trf_seq

#计算AU含量
def AU_contant(trf_feature):
    AU_list = pd.DataFrame([0]*len(trf_feature))
    AU_list.columns = ["AU_contant"]
    for i in range(len(trf_feature)):
        upscore = 0
        downscore = 0
        for j in range(30):
            if(trf_feature.iloc[i,2][j] == "A" or trf_feature.iloc[i,2][j] == "T"):
                upscore += 1/(j+1)
            if(trf_feature.iloc[i,3][j] == "A" or trf_feature.iloc[i,3][j] == "T"):
                downscore += 1/(j+1)
        AU_list.iloc[i,0] = (upscore + downscore)/2
    trf_feature = pd.concat([trf_feature,AU_list],axis = 1)
    return trf_feature

#看该位点是否匹配
def ispaired(nt):
    if nt == "A":
        return "T"
    if nt == "T":
        return "A"
    if nt == "G":
        return "C"
    if nt == "C":
        return "G"
    else:
        return ""

# 确定结合区位置
def bind_loc(trf_feature):
    start_loc = pd.DataFrame([0]*len(trf_feature))
    start_loc.columns = ["start_loc"]
    end_loc = pd.DataFrame([0]*len(trf_feature))
    end_loc.columns = ["end_loc"]
    for i in range(len(trf_feature)):
        seq = trf_feature.iloc[i,0][0]
        for j in range(len(trf_feature.iloc[i,2])):
            if ispaired(seq) == trf_feature.iloc[i,2][j]:
                start_loc.iloc[i,0] = j
                end_loc.iloc[i,0] = j + len(trf_feature.iloc[i,0])
                break
    trf_feature = pd.concat([trf_feature,start_loc],axis = 1)
    trf_feature = pd.concat([trf_feature,end_loc],axis = 1)
    return trf_feature


#统计结合位点数    
def bind_num(trf_feature):
    count_paired = pd.DataFrame([0]*len(trf_feature))
    count_paired.columns = ["count_paired"]
    for i in range(len(trf_feature)):
        count_num = 0
        for j in range(min(len(trf_feature.iloc[i,0]),len(trf_feature.iloc[i,2])-trf_feature.iloc[i,6])):
            if ispaired(trf_feature.iloc[i,0][j]) == trf_feature.iloc[i,2][trf_feature.iloc[i,6]+j]:
                count_num += 1
        count_paired.iloc[i,0] = count_num
    trf_feature = pd.concat([trf_feature,count_paired],axis = 1)
    return trf_feature

#计算字符串中最长的连续字符数
def maxDup(strs):
    n=len(strs)
    maxlen=1
    length=1
    maxstr=[]
    for i in range(n-1): 
        if strs[i]==strs[i+1]:
            length+=1
            maxstr.append(strs[i+1])
            if length>maxlen:
                maxlen=length
        else:
            length=1
    return maxlen


#计算结合区的长度
def bind_length(trf_feature):
    max_length = pd.DataFrame([0]*len(trf_feature))
    max_length.columns = ["max_length"]
    pos_longest = pd.DataFrame([0]*len(trf_feature))
    pos_longest.columns = ["pos_longest"]
    for i in range(len(trf_feature)):
        seq = ""
        for j in range(min(len(trf_feature.iloc[i,0]),len(trf_feature.iloc[i,2])-trf_feature.iloc[i,6])):
            if ispaired(trf_feature.iloc[i,0][j]) == trf_feature.iloc[i,2][trf_feature.iloc[i,6]+j]:
                seq += "T"
            else:
                seq += str(j)
        max_length.iloc[i,0] = maxDup(seq)
        pos_longest.iloc[i,0] = len(seq)
    trf_feature = pd.concat([trf_feature,max_length],axis = 1)
    trf_feature = pd.concat([trf_feature,pos_longest],axis = 1)
    return trf_feature
        
#计算3'区的结合位点数
def three_prime(trf_feature):
    three_prime = pd.DataFrame([0]*len(trf_feature))
    three_prime.columns = ["three_prime"]
    for i in range(len(trf_feature)):
        count_num = 0
        for j in range(min(7,len(trf_feature.iloc[i,2])-trf_feature.iloc[i,6])):
            if ispaired(trf_feature.iloc[i,0][j]) == trf_feature.iloc[i,2][trf_feature.iloc[i,6]+j]:
                count_num += 1
        three_prime.iloc[i,0] = count_num
    trf_feature = pd.concat([trf_feature,three_prime],axis = 1)
    return trf_feature
    
#输出trf特征列表   
def feature_calculation(trf_seq,trf_list):
    trf = create(trf_seq)
    trf_feature = pd.concat([trf,trf_list],axis = 1)
    trf_feature = AU_contant(trf_feature)
    trf_feature = bind_loc(trf_feature)
    trf_feature = bind_num(trf_feature)
    trf_feature = bind_length(trf_feature)
    trf_feature = three_prime(trf_feature)
    return trf_feature

In [ ]:
#计算特征
trf_sequence = input("please input your trf sequence:")
trf_feature = feature_calculation(trf_sequence, gene_list)

In [ ]:
#获取测试集数据
test_data = trf_feature.iloc[:,5:12]
test_data = np.array(test_data).tolist()

In [ ]:
#调用模型
import joblib
logreg = joblib.load("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\logreg.pkl")
#输入测试集数据进行预测，并输出靶标基因名
result_label = logreg.predict(test_data)
target_gene = []
for i in range(len(result_label)):
    if result_label[i] == 1:
        print(trf_feature.iloc[i,1])
        target_gene.append(trf_feature.iloc[i,1])

In [ ]:
#将靶标基因写入csv文件中
target = pd.DataFrame(columns = ["gene"],data = target_gene)
target.to_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\target_gene.csv")

# 修改版本

+ 训练三个模型

+ 调用模型时传入模型类型的参数进行模型选择调用

+ 通过调用不同模型输出不同结合区的靶基因

In [1]:
# 加载数据预处理
#upload data and package
import pandas as pd
import mglearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

trf_train_data_3 = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\train_data_3.csv")
trf_test_data_3 = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\test_data_3.csv")

trf_train_data_5 = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\train_data_5.csv")
trf_test_data_5 = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\test_data_5.csv")

trf_train_data_cds = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\train_data_cds.csv")
trf_test_data_cds = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\test_data_cds.csv")

In [2]:
#设置训练集和测试集数据
train_data_3 = trf_train_data_3[trf_train_data_3.columns[9:16]]
train_data_3 = np.array(train_data_3).tolist()

test_data_3 = trf_test_data_3[trf_test_data_3.columns[9:16]]
test_data_3 = np.array(test_data_3).tolist()


train_data_5 = trf_train_data_5[trf_train_data_5.columns[9:16]]
train_data_5 = np.array(train_data_5).tolist()

test_data_5 = trf_test_data_5[trf_test_data_5.columns[9:16]]
test_data_5 = np.array(test_data_5).tolist()


train_data_cds = trf_train_data_cds[trf_train_data_cds.columns[9:16]]
train_data_cds = np.array(train_data_cds).tolist()

test_data_cds = trf_test_data_cds[trf_test_data_cds.columns[9:16]]
test_data_cds = np.array(test_data_cds).tolist()

In [4]:
#设置标签数据
train_data_label = np.array(trf_train_data_3).tolist()
train_data_label = [i[16] for i in train_data_label]

test_data_label = np.array(trf_test_data_3).tolist()
test_data_label = [i[16] for i in test_data_label]

In [5]:
X_train_3 = train_data_3
X_test_3 = test_data_3

X_train_5 = train_data_5
X_test_5 = test_data_5

X_train_cds = train_data_cds
X_test_cds = test_data_cds

y_train = train_data_label
y_test = test_data_label

In [6]:
#使用逻辑回归模型(0.1)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5,shuffle=True, random_state=0)
#正则化系数C = 1时(可调节参数C)
logreg_3 = LogisticRegression(C = 0.001).fit(X_train_3, y_train)
logreg_5 = LogisticRegression(C = 0.001).fit(X_train_5, y_train)
logreg_cds = LogisticRegression(C = 0.001).fit(X_train_cds, y_train)


print("logreg_3 Training set score: {:.3f}".format(logreg_3.score(X_train_3, y_train)))
print("logreg_3 Test set score: {:.3f}".format(logreg_3.score(X_test_3, y_test)))
print("logreg_3 The feature weight: {}".format(logreg_3.coef_))

print("logreg_5 Training set score: {:.3f}".format(logreg_5.score(X_train_5, y_train)))
print("logreg_5 Test set score: {:.3f}".format(logreg_5.score(X_test_5, y_test)))
print("logreg_5 The feature weight: {}".format(logreg_5.coef_))


print("logreg_cds Training set score: {:.3f}".format(logreg_cds.score(X_train_cds, y_train)))
print("logreg_cds Test set score: {:.3f}".format(logreg_cds.score(X_test_cds, y_test)))
print("logreg_cds The feature weight: {}".format(logreg_cds.coef_))

logreg_3 Training set score: 0.541
logreg_3 Test set score: 0.538
logreg_3 The feature weight: [[-0.27364661  0.00758203 -0.00074106  0.00289397  0.01080048 -0.00191381
  -0.03828901]]
logreg_5 Training set score: 0.542
logreg_5 Test set score: 0.538
logreg_5 The feature weight: [[-0.25724275  0.01541082 -0.006221    0.02890959 -0.02306304 -0.00137002
  -0.01695497]]
logreg_cds Training set score: 0.541
logreg_cds Test set score: 0.544
logreg_cds The feature weight: [[-0.27744316  0.01470373 -0.00667913  0.02300122  0.00409262 -0.00332068
  -0.00563413]]


In [7]:
#保存模型
import joblib
joblib.dump(logreg_3,"E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\logreg_3.pkl")

joblib.dump(logreg_5,"E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\logreg_5.pkl")

joblib.dump(logreg_cds,"E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\logreg_cds.pkl")


['E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\logreg_cds.pkl']

In [8]:
#读取基因列表
gene_list = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\unique_gene.csv")
gene_list = gene_list.iloc[:,1:5]

In [10]:
#定义特征计算函数

#1.创建一个trf序列的列表
def create(trf,gene_list):
    seq = [trf]
    trf_seq = pd.DataFrame(seq*len(gene_list))
    trf_seq.columns = ["trf_seq"]
    return trf_seq

#计算AU含量
def AU_contant(trf_feature):
    AU_list = pd.DataFrame([0]*len(trf_feature))
    AU_list.columns = ["AU_contant"]
    for i in range(len(trf_feature)):
        upscore = 0
        downscore = 0
        for j in range(30):
            if(trf_feature.iloc[i,2][j] == "A" or trf_feature.iloc[i,2][j] == "T"):
                upscore += 1/(j+1)
            if(trf_feature.iloc[i,3][j] == "A" or trf_feature.iloc[i,3][j] == "T"):
                downscore += 1/(j+1)
        AU_list.iloc[i,0] = (upscore + downscore)/2
    trf_feature = pd.concat([trf_feature,AU_list],axis = 1)
    return trf_feature

#看该位点是否匹配
def ispaired(nt):
    if nt == "A":
        return "T"
    if nt == "T":
        return "A"
    if nt == "G":
        return "C"
    if nt == "C":
        return "G"
    else:
        return ""

# 确定结合区位置
def bind_loc(trf_feature,index):
    start_loc = pd.DataFrame([0]*len(trf_feature))
    start_loc.columns = ["start_loc"]
    end_loc = pd.DataFrame([0]*len(trf_feature))
    end_loc.columns = ["end_loc"]
    for i in range(len(trf_feature)):
        seq = trf_feature.iloc[i,0][0]
        for j in range(len(trf_feature.iloc[i,index+1])):
            if ispaired(seq) == trf_feature.iloc[i,index+1][j]:
                start_loc.iloc[i,0] = j
                end_loc.iloc[i,0] = j + len(trf_feature.iloc[i,0])
                break
    trf_feature = pd.concat([trf_feature,start_loc],axis = 1)
    trf_feature = pd.concat([trf_feature,end_loc],axis = 1)
    return trf_feature


#统计结合位点数    
def bind_num(trf_feature,index):
    count_paired = pd.DataFrame([0]*len(trf_feature))
    count_paired.columns = ["count_paired"]
    for i in range(len(trf_feature)):
        count_num = 0
        for j in range(min(len(trf_feature.iloc[i,0]),len(trf_feature.iloc[i,index+1])-trf_feature.iloc[i,6])):
            if ispaired(trf_feature.iloc[i,0][j]) == trf_feature.iloc[i,index+1][trf_feature.iloc[i,6]+j]:
                count_num += 1
        count_paired.iloc[i,0] = count_num
    trf_feature = pd.concat([trf_feature,count_paired],axis = 1)
    return trf_feature

#计算字符串中最长的连续字符数
def maxDup(strs):
    n=len(strs)
    maxlen=1
    length=1
    maxstr=[]
    for i in range(n-1): 
        if strs[i]==strs[i+1]:
            length+=1
            maxstr.append(strs[i+1])
            if length>maxlen:
                maxlen=length
        else:
            length=1
    return maxlen


#计算结合区的长度
def bind_length(trf_feature,index):
    max_length = pd.DataFrame([0]*len(trf_feature))
    max_length.columns = ["max_length"]
    pos_longest = pd.DataFrame([0]*len(trf_feature))
    pos_longest.columns = ["pos_longest"]
    for i in range(len(trf_feature)):
        seq = ""
        for j in range(min(len(trf_feature.iloc[i,0]),len(trf_feature.iloc[i,index+1])-trf_feature.iloc[i,6])):
            if ispaired(trf_feature.iloc[i,0][j]) == trf_feature.iloc[i,index+1][trf_feature.iloc[i,6]+j]:
                seq += "T"
            else:
                seq += str(j)
        max_length.iloc[i,0] = maxDup(seq)
        pos_longest.iloc[i,0] = len(seq)
    trf_feature = pd.concat([trf_feature,max_length],axis = 1)
    trf_feature = pd.concat([trf_feature,pos_longest],axis = 1)
    return trf_feature
        
#计算3'区的结合位点数
def three_prime(trf_feature):
    three_prime = pd.DataFrame([0]*len(trf_feature))
    three_prime.columns = ["three_prime"]
    for i in range(len(trf_feature)):
        count_num = 0
        for j in range(min(7,len(trf_feature.iloc[i,2])-trf_feature.iloc[i,6])):
            if ispaired(trf_feature.iloc[i,0][j]) == trf_feature.iloc[i,2][trf_feature.iloc[i,6]+j]:
                count_num += 1
        three_prime.iloc[i,0] = count_num
    trf_feature = pd.concat([trf_feature,three_prime],axis = 1)
    return trf_feature
    
#输出trf特征列表   
def feature_calculation(trf_seq,trf_list,index):
    trf = create(trf_seq,trf_list)
    trf_feature = pd.concat([trf,trf_list],axis = 1)
    trf_feature = AU_contant(trf_feature)
    trf_feature = bind_loc(trf_feature,index)
    trf_feature = bind_num(trf_feature,index)
    trf_feature = bind_length(trf_feature,index)
    trf_feature = three_prime(trf_feature)
    return trf_feature

In [11]:
#计算特征(先输入要预测的trf序列，)
trf_sequence = input("please input your trf sequence:")
index = int(input("please input the type of bind region:"))
trf_feature = feature_calculation(trf_sequence, gene_list,index)

please input your trf sequence:AGCTCGGTTCGATATGCGGCCCTTTA
please input the type of bind region:2


In [12]:
#获取测试集数据
test_data = trf_feature.iloc[:,5:12]
test_data = np.array(test_data).tolist()

In [13]:
#调用模型
import joblib
logreg_3 = joblib.load("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\logreg_3.pkl")
logreg_5 = joblib.load("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\logreg_5.pkl")
logreg_cds = joblib.load("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\logreg_cds.pkl")
#输入测试集数据进行预测，并输出靶标基因名
if index == 1:
    result_label = logreg_3.predict(test_data)
if index == 2:
    result_label = logreg_5.predict(test_data)
if index == 3:
    result_label = logreg_cds.predict(test_data)

target_gene = []
for i in range(len(result_label)):
    if result_label[i] == 1:
        print(trf_feature.iloc[i,1])
        target_gene.append(trf_feature.iloc[i,1])

ABCB4
ABCC1
ABCC8
ABHD17A
ACO1
ACTN4
ACTR2
ADAM11
ADAM15
ADAMTS10
ADAMTSL3
ADARB1
ADGB
ADGRA2
ADGRB2
ADI1
ADRA1D
ADRM1
AEN
AFDN
AFF3
AGBL5
AGPAT2
AIRE
AJAP1
AKT1
ALDH1A2
ALPK3
ALPL
AMFR
AMMECR1L
AMOTL1
ANAPC1
ANHX
ANKRD52
ANKRD63
ANKS1B
ANXA6
AP1B1
AP1M1
AP2A1
AP5B1
APBA1
APPBP2
ARFRP1
ARHGAP10
ARHGAP23
ARHGAP26
ARHGAP35
ARHGAP39
ARHGAP4
ARHGAP6
ARHGEF10L
ARHGEF37
ARHGEF40
ARID3A
ARID3B
ARL16
ARL6IP4
ARNT2
ARRB1
ARSA
ASIC2
ASIC4
ATAD3B
ATG101
ATG16L2
ATP1A1
ATRN
ATXN7L2
AURKAIP1
B3GALT4
B4GALT7
B9D2
BAALC
BAHCC1
BAIAP2L2
BAIAP3
BAMBI
BANP
BAX
BCAR1
BEAN1
BHLHA9
BICD2
BICRA
BIN1
BIRC5
BLM
BMP8A
BPIFB1
BRD2
BRD7
BTG2
BTG3
C11orf49
C11orf80
C11orf96
C16orf95
C17orf113
C1orf159
C1orf216
C1QL2
C1QTNF12
C1QTNF8
C20orf204
C2CD2L
C2orf16
C2orf72
C3orf70
C6orf47
C8orf82
C9orf153
CA3
CA7
CABLES1
CABP1
CABP7
CACNA1B
CACNA1D
CACNA1H
CACNA2D2
CACNG8
CACTIN
CADM4
CALML3
CALN1
CAMK1D
CAP1
CAPG
CAPS2
CARD19
CASKIN1
CASP7
CASTOR1
CBX1
CCBE1
CCDC113
CCDC124
CCDC178
CCDC71L
CCDC74A
CCDC74B
CCDC92
CCDC93


In [ ]:
#将靶标基因写入csv文件中
target = pd.DataFrame(columns = ["gene"],data = target_gene)
target.to_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\target_gene.csv")